In [1]:
import math
from dataclasses import dataclass
from typing import Any, Callable, Dict, List, NewType, Optional, Tuple, Union, Mapping
from abc import ABC, abstractmethod

import datasets
import torch
import torch.nn.functional as F
import numpy as np
import torch.nn as nn
import logging
import copy
import gc

from datasets import load_dataset
from tqdm import tqdm
from transformers import (
    PreTrainedTokenizerBase,
    PreTrainedModel,
    PretrainedConfig,
    AutoConfig,
    AutoModelForCausalLM,
    LlamaForCausalLM,
    LlamaConfig,
    Trainer,
    TrainingArguments,
    AutoTokenizer,
    HfArgumentParser,
    default_data_collator,
    is_torch_xla_available,
    set_seed,
)
from transformers import (
    HfArgumentParser,
    TrainingArguments,
    Trainer
)

from transformers.modeling_outputs import (
    BaseModelOutputWithPast,
    CausalLMOutputWithPast
)

import efficient_masks
import accurate_masks

from utils import (
    generate, 
    get_hidden_states, 
    get_logits,
    free_memory
)
# Configure logger
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

In [2]:
path_a = "unsloth/Llama-3.2-1B-Instruct"
path_b = "unsloth/Llama-3.2-1B"
merge_config_a = accurate_masks.MergerConfig(
    model_paths = [path_a, path_b],
    mode = "vector_input",
    constrain_mode = "01",
)
merge_config_e = efficient_masks.MergerConfig(
    model_paths = [path_a, path_b],
    mode = "vector_input",
    constrain_mode = "01",
)

In [3]:
merge_config_a

MergerConfig {
  "constrain_mode": "01",
  "mode": "vector_input",
  "model_paths": [
    "unsloth/Llama-3.2-1B-Instruct",
    "unsloth/Llama-3.2-1B"
  ],
  "transformers_version": "4.46.3"
}

In [4]:
def debug_linear_forward(self, x):
    constrained_weight_masks = self.weight_masks_constrainer([m.weight for m in self.weight_masks])
    constrained_bias_masks = self.bias_masks_constrainer(
        [m.weight if m is not None else None for m in self.bias_masks]
    )
    masked_biases = [
        b_mask * linear.bias if linear.bias is not None and b_mask is not None else linear.bias
        for b_mask, linear in zip(constrained_bias_masks, self.linears)
    ]
    merged_bias = (
        sum(b if b is not None else torch.zeros_like(
            self.linears[0].weight[:, 0]) for b in masked_biases
           ) 
        if not all(b is None for b in masked_biases) else None
    )

    logger.info("Debugging Linear forward.")
    output = 0.0
    for i, linear in enumerate(self.linears):
        logger.info(f"BEFORE")
        logger.info(f"  linear: device: {linear.weight.device}; dtype: {linear.weight.dtype}")
        logger.info(f"  input: device: {x.device}; dtype: {x.dtype}")
        masked_input = constrained_weight_masks[i] * x
        logger.info(f"AFTER")
        logger.info(f"  linear: device: {linear.weight.device}; dtype: {linear.weight.dtype}")
        logger.info(f"  input: device: {masked_input.device}; dtype: {masked_input.dtype}")
        output = output + nn.functional.linear(masked_input, linear.weight, None)
        logger.info(f"OUTPUT")
        logger.info(f"  output: device: {output.device}; dtype: {output.dtype}")
    if merged_bias:
        output = output + merged_bias

    return output

In [5]:
def debug_emb_forward(self, input_ids):
    constrained_masks = self.masks_constrainer([m.weight for m in self.masks])
    logger.info("Debugging Embedding forward.")
    an_embedding = self.embeddings[0]
    out = 0.0
    for i, emb in enumerate(self.embeddings):
        logger.info(f"BEFORE")
        logger.info(f"  emb: device: {emb.weight.device}; dtype: {emb.weight.dtype}")
        logger.info(f"  input: device: {input_ids.device}; dtype: {input_ids.dtype}")
        mask = constrained_masks[i]
        masked_weight = emb.weight * mask
        logger.info(f"  mask: device: {mask.device}; dtype: {mask.dtype}")
        logger.info(f"  masked_emb: device: {masked_weight.device}; dtype: {masked_weight.dtype}")
        out = out + nn.functional.embedding(
            input_ids,
            # emb.weight * mask,
            masked_weight,
            padding_idx=an_embedding.padding_idx,
            max_norm=an_embedding.max_norm,
            norm_type=an_embedding.norm_type,
            scale_grad_by_freq=an_embedding.scale_grad_by_freq,
            sparse=an_embedding.sparse,
        )
        logger.info(f"AFTER")
        logger.info(f"  output: device: {out.device}; dtype: {out.dtype}")
    return out

In [5]:
# efficient_masks.LinearsWithMasks.forward = debug_linear_forward
# efficient_masks.EmbeddingsWithMasks.forward = debug_emb_forward

In [6]:
tokenizer = AutoTokenizer.from_pretrained(path_a)

In [7]:
em = efficient_masks.Merger(merge_config_e)
em.__post_init__()

Initializing masks: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 147/147 [00:48<00:00,  3.01it/s]
2025-01-03 11:28:12,554 - INFO - Initial GPU memory allocated: 0.00 GB
2025-01-03 11:28:12,941 - INFO - Final GPU memory allocated: 0.00 GB
2025-01-03 11:28:12,942 - INFO - Freed GPU memory: 0.00 GB


In [8]:
em = em.to(device="cuda:0", dtype=torch.bfloat16)

In [9]:
am = accurate_masks.Merger(merge_config_a)
am.__post_init__()

Initializing masks: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 147/147 [00:44<00:00,  3.32it/s]
2025-01-03 11:29:02,641 - INFO - Initial GPU memory allocated: 4.61 GB
2025-01-03 11:29:03,009 - INFO - Final GPU memory allocated: 4.61 GB
2025-01-03 11:29:03,013 - INFO - Freed GPU memory: 0.00 GB


In [10]:
am = am.to(device="cuda:1", dtype=torch.bfloat16)

In [11]:
prompt = "How to attack a person with an egg. Talk like a crazy person."
logits_merged_a = get_logits(prompt, am.merger, tokenizer)
logits_merged_e = get_logits(prompt, em.merger, tokenizer)

In [12]:
logits_merged_a = logits_merged_a.to(logits_merged_e.device)

In [13]:
logits_merged_a.shape

torch.Size([1, 16, 128256])

In [14]:
probs_a = torch.softmax(logits_merged_a, dim=-1)
probs_e = torch.softmax(logits_merged_e, dim=-1)

In [15]:
torch.topk(probs_a[0, 0, :], k=20)

torch.return_types.topk(
values=tensor([6.4453e-01, 3.0469e-01, 2.5024e-02, 1.1841e-02, 3.3875e-03, 2.0599e-03,
        9.7275e-04, 9.7275e-04, 7.5531e-04, 3.5667e-04, 3.5667e-04, 2.7847e-04,
        2.1648e-04, 1.9169e-04, 1.6880e-04, 1.6880e-04, 1.4877e-04, 1.4877e-04,
        1.3161e-04, 1.3161e-04], device='cuda:0', dtype=torch.bfloat16),
indices=tensor([  755,     2,   791, 16309,   475,  3936,    17,    16,    32,    59,
         1527,    11,    51,    50,  1687, 13066,  2028,    34,   220,    35],
       device='cuda:0'))

In [16]:
torch.topk(probs_e[0, 0, :], k=20)

torch.return_types.topk(
values=tensor([6.9531e-01, 2.5586e-01, 2.0996e-02, 1.2756e-02, 4.6997e-03, 2.2125e-03,
        1.0452e-03, 1.0452e-03, 8.1635e-04, 6.3324e-04, 3.8528e-04, 3.8528e-04,
        2.3365e-04, 2.3365e-04, 1.8215e-04, 1.8215e-04, 1.6022e-04, 1.4210e-04,
        1.4210e-04, 1.4210e-04], device='cuda:0', dtype=torch.bfloat16),
indices=tensor([  755,     2,   791, 16309,   475,  3936,    17,    16,    32,    59,
         1527,    11,    50,    51,   220, 13066,  1687,    34,    35,    40],
       device='cuda:0'))

In [17]:
def count_trainable_params(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [18]:
count_trainable_params(em.merger)

663618

In [19]:
count_trainable_params(am.merger)

663618

In [1]:
"""
Model merging training implementation using PyTorch and Transformers.
Implements custom data collation and training for merged language models.
"""
import math
from dataclasses import dataclass
from typing import Any, Callable, Dict, List, NewType, Optional, Tuple, Union, Mapping
from abc import ABC, abstractmethod

import datasets
import torch
import torch.nn.functional as F
import safetensors
import numpy as np
import torch.nn as nn
import logging
import copy
import gc

from datasets import load_dataset
from tqdm import tqdm
from transformers import (
    PreTrainedTokenizerBase,
    PreTrainedModel,
    PretrainedConfig,
    AutoConfig,
    AutoModelForCausalLM,
    LlamaForCausalLM,
    LlamaConfig,
    Trainer,
    TrainingArguments,
    AutoTokenizer,
    HfArgumentParser,
    default_data_collator,
    is_torch_xla_available,
    set_seed,
)

from transformers.utils import CONFIG_NAME

from merger import (
# from efficient_masks import (
    MergerConfig,
    # Merger,
    NewMerger,
    init_masks,
    set_masks
)

from utils import (
    generate, 
    get_hidden_states, 
    get_logits,
    free_memory
)
# Configure logger
logging.basicConfig(
    level=logging.INFO, 
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

In [2]:
import os
# Option 1: Set specific GPU devices
os.environ["CUDA_VISIBLE_DEVICES"] = "7" 

In [3]:
checkpoint_dir = "./trained_masks/checkpoint-300/"

In [4]:
merge_config = MergerConfig.from_pretrained(checkpoint_dir)
merge_config

MergerConfig {
  "architectures": [
    "NewMerger"
  ],
  "constrain_mode": "01",
  "mode": "vector_input",
  "model_paths": [
    "nguyenthanhdo/llama32_smol_rewrite_50k",
    "nguyenthanhdo/llama32_smol_summarize_50k"
  ],
  "torch_dtype": "bfloat16",
  "transformers_version": "4.46.3"
}

In [5]:
merger = NewMerger.from_pretrained(
    checkpoint_dir,
    merge_config,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    attn_implementation="flash_attention_2",
)

2025-01-06 12:02:09,191 - INFO - Creating merger with dummy weights ...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Initializing masks: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 255/255 [00:27<00:00,  9.17it/s]


In [7]:
merger.merger.model.layers[21].mlp.up_proj.get_constrained_masks()

{'weight_masks': [tensor([0.5156, 0.5000, 0.5117,  ..., 0.4668, 0.4727, 0.5000],
         dtype=torch.bfloat16),
  tensor([0.5156, 0.5000, 0.5117,  ..., 0.4688, 0.4707, 0.5000],
         dtype=torch.bfloat16)],
 'bias_masks': [None, None]}

In [2]:
# # Initialize configuration
# merge_config = MergerConfig(
#     model_paths=[
#         "nguyenthanhdo/llama32_smol_rewrite_50k",
#         "nguyenthanhdo/llama32_smol_summarize_50k",
#     ],
#     mode="vector_input",
#     constrain_mode="identity"
# )

# merger = NewMerger.from_pretrained(
#     None,
#     merge_config,
#     torch_dtype=torch.bfloat16,
#     device_map="auto",
#     attn_implementation="flash_attention_2",
# )
# set_masks(merger.merger, strategy="uniform", factors=[0.95, 0.05])

2025-01-06 07:58:21,126 - INFO - Creating merger with dummy weights ...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Setting up masks: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 255/255 [00:00<00:00, 39634.89it/s]


In [8]:
merger = merger.to(device="cuda:7")

In [9]:
tokenizer = AutoTokenizer.from_pretrained(merge_config.model_paths[0])
tokenizer.pad_token = tokenizer.eos_token

In [10]:
class MergerTrainer(Trainer):
    """Custom trainer for merged model training."""
    
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.pop("labels")
        data_source = inputs.pop("data_source")
        effective_idxs = (labels != -100).float().unsqueeze(dim=-1)
        
        outputs = model(**inputs)
        logits_merged = outputs["merger_outputs"].logits
        logits_components = [x.logits for x in outputs["components_outputs"]]

        # Compute target logits and KL divergence
        logits_target = selective_logits_target(logits_components, data_source)
        temperature = 1.0
        kl_fct = nn.KLDivLoss(reduction="none")
        diff = (
            kl_fct(
                F.log_softmax(logits_target / temperature, dim=-1),
                F.softmax(logits_merged / temperature, dim=-1)
            )
            * (temperature) ** 2
        )
        
        # Calculate final loss
        loss = (diff * effective_idxs).sum(dim=-1)
        loss = (loss / effective_idxs.sum(dim=1)).mean()

        return (loss, outputs) if return_outputs else loss

In [11]:
summarize_dataset = load_dataset(
    "HuggingFaceTB/smoltalk",
    "smol-summarize",
    split="train"
)
rewrite_dataset = load_dataset(
    "HuggingFaceTB/smoltalk",
    "smol-rewrite",
    split="train"
)

In [36]:
idx = 50000
train_dataset = summarize_dataset
system = train_dataset[idx]['messages'][0]['content']
prompt = train_dataset[idx]['messages'][1]['content']
messages = [
    {"role": "system", "content": system},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    # train_dataset[idx]['messages'],
    messages,
    tokenize=False,
    add_generation_prompt=True
)
print(text)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Extract and present the main key point of the input text in one very short sentence, including essential details like dates or locations if necessary.<|eot_id|><|start_header_id|>user<|end_header_id|>

Hi Naomi,

I hope you're having a good week so far. I wanted to share some exciting news with you - I just found out that our research project has been awarded a major grant from the National Health and Medical Research Council!

This funding will allow us to expand our study and collect more comprehensive data on the social determinants of health in Indigenous Australian communities. It's a huge vote of confidence in our work and a testament to the importance of the issues we're addressing.

I wanted to let you know right away, as I think this could have significant implications for our collaboration going forward. With this additional funding, we'll be able to take on a larger-scale project and potentially make an even greate

In [40]:
answer = generate(text, merger.models[1], tokenizer)

Liam shares that the research project has received a major grant from the National Health and Medical Research Council, allowing for expanded study and data collection. Liam is eager to discuss how to leverage this funding to enhance the project's impact. Liam is excited to celebrate the news and hear feedback.<|end_of_text|>


In [38]:
answer = generate(text, merger.merger, tokenizer)

Liam shares that the research project has received a major grant from the National Health and Medical Research Council, enabling expanded data collection on social determinants of health in Indigenous Australian communities. Liam is eager to discuss how to leverage this funding to enhance the project's impact.<|end_of_text|>


In [54]:
def selective_logits_target(logits_components, data_source):
    """Select appropriate logits based on data source."""
    # logits_components ~ [(batch_size, seq_len, vocab_size) * n_components]
    # stacked_logits.shape = (n_components, batch_size, seq_len, vocab_size)
    # data_source.shape == (batch_size,)
    # indices.shape == (batch_size, 1, 1)
    stacked_logits = torch.stack(logits_components)
    indices = data_source.unsqueeze(-1).unsqueeze(-1)
    return stacked_logits[indices]

In [52]:
logits_merged = get_logits(text, merger.merger, tokenizer)
logits_a = get_logits(text, merger.models[0], tokenizer)
logits_b = get_logits(text, merger.models[1], tokenizer)

In [33]:
logits_merged.shape

torch.Size([1, 241, 128256])

In [1]:
def masked_kl_div(logits_a, logits_b, mask):
    logits_a = logits_a.view(-1, logits_a.size(-1))
    logits_b = logits_b.view(-1, logits_b.size(-1))
    mask = mask.view(-1)

    assert mask.size(0) == logits_a.size(0)

    log_probs_a = nn.functional.log_softmax(logits_a, dim=-1)
    log_probs_b = nn.functional.log_softmax(logits_b, dim=-1)

    div = log_probs_a.exp() * (log_probs_a - log_probs_b)
    div = div.sum(-1)

    div = (div * mask).sum() / mask.sum()
    return div

In [26]:
def kl(logits_a, logits_b):
    probs_a = nn.functional.softmax(logits_a, dim=-1)
    
    loss = probs_a * (probs_a.log() - nn.functional.log_softmax(logits_b, dim=-1))
    return loss.sum(-1).mean()

tensor(0.6875, device='cuda:0', dtype=torch.bfloat16)